
> We will undestand how to deal with the problem of curse of dimesnioanlity using PCA and will also see how to implement principal component analysis using python. 


**Data Set**: We will make use of vehicle-2.csv data set 
The data contains features extracted from the silhouette of vehicles in
different angles. Four Corgie & model vehicles were used for the
experiment: a double decker bus, Cheverolet van, Saab 9000 and an
Opel Manta 400 cars. This particular combination of vehicles was
chosen with the expectation that the bus, van and either one of the cars
would be readily distinguishable, but it would be more difficult to
distinguish between the cars.

**Objective:**

The purpose is to classify a given silhouette as one of three types of
vehicle, using a set of features extracted from the silhouette. The vehicle
may be viewed from one of many different angles.
How We Will Meet Our Objective?
We will Apply dimensionality reduction technique - PCA and train a model using the reduced set of principal components (Attributes/dimension). Then we will build Support Vector Classifier on raw data and also on PCA components to see how the model perform on the reduced set of dimension. We will also print the confusion matrix for both the scenario and see how our model has performed in classifying various vehicle types based on given silhouette of vehicles.

**Let's Go Hands-On:**

In [ ]:
#let us start by importing the relevant libraries

%matplotlib inline
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')
#import the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report,roc_auc_score
from scipy.stats import zscore
from sklearn.model_selection import train_test_split


## Loading Data Set

In [ ]:
vehdf= pd.read_csv("../input/vehicle-2.csv")
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#print(vehdf)
vehdf.head(200)


In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
le = LabelEncoder() 
columns = vehdf.columns
#Let's Label Encode our class variable: 
print(columns)
vehdf['class'] = le.fit_transform(vehdf['class'])
vehdf.shape

# Performing EDA: 

  - Finding Any Missing Value
  - Finding Outliers
  - Understanding attributes using descriptive statistics
  - visualizing attribute distribution using univariate and multivariate analysis
  - Finding attribute correlation and analysing which attribute is more important

## Quick Eyeballing To See if there is any missing values

In [ ]:
vehdf.info()

## Quick Insights: 
 We can see that :
    - circularity, class, hollow_ratio,max.length_rectangularity, , max.length_aspect_ratio, compactness has no missing values rest all features are having some kind of missing values 
    - All attributes are of numerical type
 # Finding The Missing Value: 
   Let's find the count of each attribite & treat the missing values   

In [ ]:
from sklearn.impute import SimpleImputer

newdf = vehdf.copy()

X = newdf.iloc[:,0:19] #separting all numercial independent attribute
#y = vehdf.iloc[:,18] #seprarting class attribute. 
#imputer = SimpleImputer()
imputer = SimpleImputer(missing_values=np.nan, strategy='median', verbose=1)
#fill missing values with mean column values
transformed_values = imputer.fit_transform(X)
column = X.columns
print(column)
newdf = pd.DataFrame(transformed_values, columns = column )
newdf.describe()


In [ ]:


print("Original null value count:", vehdf.isnull().sum())
print("\n\nCount after we impiuted the NaN value: ", newdf.isnull().sum())

### Obseravtion: 

If you carefully observe above, our orginal dataframe vehdf and new dataframe df , we will find that , After we imputed the datfarme series , using simpleimputer, we can see that the missing NaN values from our orginal vehdf datframe columns are treated and replaced using mode strategy. 

# Understanding each attributes : 

Univariate 

  - Quick descriptive statistics to make some meaningful sense of data 
  - Plotting univariate distribution
  - Finding outliers & skewness in data series.  
  - Treating outliers
   

## Descriptive statistical summary 

describe() Function gives the mean, std and IQR values. It excludes character column and calculate summary statistics only for numeric columns.

In [ ]:
newdf.describe().T

In [ ]:
newdf.shape

### Quick Insights On descriptive stats: 
   -  Compactness has mean and median values almost similar , it signifies that it is normally distribited and has no skewness/outlier 
   - circularity : it also seems to be normally distribted as mean amd median has similar values
   - scatter_ratio feature seems to be having some kind of skewness and outlier
   - Scaled variance 1 & 2 

In [ ]:
plt.style.use('seaborn-whitegrid')

newdf.hist(bins=20, figsize=(60,40), color='lightblue', edgecolor = 'red')
plt.show()

Quick Observation : 
   - Most of the data attributes seems to be normally distributed 
   - scaled valriance 1 and skewness about 1 and 2, scatter_ratio, seems to be right skwed .
   - pr.axis_rectangularity seems to be haing outliers as there are some gaps found in the bar plot. 

In [ ]:


#Let us use seaborn distplot to analyze the distribution of our columns and see the skewness in attributes
f, ax = plt.subplots(1, 6, figsize=(30,5))
vis1 = sns.distplot(newdf["scaled_variance.1"],bins=10, ax= ax[0])
vis2 = sns.distplot(newdf["scaled_variance"],bins=10, ax=ax[1])
vis3 = sns.distplot(newdf["skewness_about.1"],bins=10, ax= ax[2])
vis4 = sns.distplot(newdf["skewness_about"],bins=10, ax=ax[3])
vis6 = sns.distplot(newdf["scatter_ratio"],bins=10, ax=ax[5])

f.savefig('subplot.png')

  

In [ ]:
skewValue = newdf.skew()
print("skewValue of dataframe attributes: ", skewValue)

# Univariate Analysis Using Boxplot:

In descriptive statistics, a box plot is a method for graphically depicting groups of numerical data through their quartiles. Box plots may also have lines extending vertically from the boxes (whiskers) indicating variability outside the upper and lower quartiles, hence the terms box-and-whisker plot and box-and-whisker diagram. Outliers may be plotted as individual points.


In [ ]:
#Summary View of all attribute , The we will look into all the boxplot individually to trace out outliers

ax = sns.boxplot(data=newdf, orient="h")


In [ ]:
plt.figure(figsize= (20,15))
plt.subplot(3,3,1)
sns.boxplot(x= newdf['pr.axis_aspect_ratio'], color='orange')

plt.subplot(3,3,2)
sns.boxplot(x= newdf.skewness_about, color='purple')

plt.subplot(3,3,3)
sns.boxplot(x= newdf.scaled_variance, color='brown')

plt.show()



In [ ]:
plt.figure(figsize= (20,15))
plt.subplot(3,3,1)
sns.boxplot(x= newdf['radius_ratio'], color='red')

plt.subplot(3,3,2)
sns.boxplot(x= newdf['scaled_radius_of_gyration.1'], color='lightblue')

plt.subplot(3,3,3)
sns.boxplot(x= newdf['scaled_variance.1'], color='yellow')

plt.show()

In [ ]:
plt.figure(figsize= (20,15))
plt.subplot(3,3,1)
sns.boxplot(x= newdf['max.length_aspect_ratio'], color='green')

plt.subplot(3,3,2)
sns.boxplot(x= newdf['skewness_about.1'], color='grey')


plt.show()

# Observation on boxplots: 
   - pr.axis_aspect_ratio, skewness_about, max_length_aspect_ratio, skewness_about_1, 
   - scaled_radius_of_gyration.1, scaled_variance.1, radius_ratio, skewness_about, scaled_variance.1 
 are some of the attributes with outliers. which is visible with all dotted points   

## Treating Outliers Using IQR: Upper whisker

The interquartile range (IQR), also called the midspread or middle 50%, or technically H-spread, is a measure of statistical dispersion, being equal to the difference between 75th and 25th percentiles, or between upper and lower quartiles, IQR = Q3 − Q1.
  

In [ ]:
newdf.shape


### As we now have the IQR scores, it’s time to get hold on outliers. The below code will give an output with some true and false values. The data point where we have False that means these values are valid whereas True indicates presence of an outlier.

In [ ]:
from scipy.stats import iqr

Q1 = newdf.quantile(0.25)
Q3 = newdf.quantile(0.75)
IQR = Q3 - Q1
print(IQR)




In [ ]:
cleandf = newdf[~((newdf < (Q1 - 1.5 * IQR)) |(newdf > (Q3 + 1.5 * IQR))).any(axis=1)]
cleandf.shape

## Let's Plot The Box Plot Once Agaian To See if outliers are removed.

In [ ]:
plt.figure(figsize= (20,15))
plt.subplot(8,8,1)
sns.boxplot(x= cleandf['pr.axis_aspect_ratio'], color='orange')

plt.subplot(8,8,2)
sns.boxplot(x= cleandf.skewness_about, color='purple')

plt.subplot(8,8,3)
sns.boxplot(x= cleandf.scaled_variance, color='brown')
plt.subplot(8,8,4)
sns.boxplot(x= cleandf['radius_ratio'], color='red')

plt.subplot(8,8,5)
sns.boxplot(x= cleandf['scaled_radius_of_gyration.1'], color='lightblue')

plt.subplot(8,8,6)
sns.boxplot(x= cleandf['scaled_variance.1'], color='yellow')

plt.subplot(8,8,7)
sns.boxplot(x= cleandf['max.length_aspect_ratio'], color='lightblue')

plt.subplot(8,8,8)
sns.boxplot(x= cleandf['skewness_about.1'], color='pink')

plt.show()


### Quick Comment: 

We can see that all out boxplot for all the attributes which had outlier have been treate and removed. Since no. of outliers were less we opted to remove it. Generally we avoid this as it can lead to info loss in case of large data sets with large no of outliers

# Understanding the relationship between all independent attribute:

We will be using data correlation: 

Data Correlation: Is a way to understand the relationship between multiple variables and attributes in your dataset. Using Correlation, you can get some insights such as:

- One or multiple attributes depend on another attribute or a cause for another attribute.
- One or multiple attributes are associated with other attributes.

Spearman and Pearson are two statistical methods to calculate the strength of correlation between two variables or attributes. Pearson Correlation Coefficient can be used with continuous variables that have a linear relationship.

## Pearson Correlation Coefficient: 

We will use Pearson Correlation Coefficient to see what all attributes are linearly related and also visualize the same in the seaborns scatter plot. 


In [ ]:
def correlation_heatmap(dataframe,l,w):
    #correlations = dataframe.corr()
    correlation = dataframe.corr()
    plt.figure(figsize=(l,w))
    sns.heatmap(correlation, vmax=1, square=True,annot=True,cmap='viridis')
    plt.title('Correlation between different fearures')
    plt.show();
    
# Let's Drop Class column and see the correlation Matrix & Pairplot Before using this dataframe for PCA as PCA should only be perfromed on independent attribute
cleandf= newdf.drop('class', axis=1)
#print("After Dropping: ", cleandf)
correlation_heatmap(cleandf, 30,15)

## Quick Insights : From Correlation Hetamap: 

### Strong/fare Correlation: 

          - Scaled Variance & Scaled Variance.1 seems to be strongly correlated with value of 0.98
          - skewness_about_2 and hollow_ratio seems to be strongly correlated, corr coeff: 0.89
          - ditance_circularity and radius_ratio seems to have high positive correlation with corr coeff: 0.81
          - compactness & circularity , radius_ratio & pr.axis_aspect_ratio also seems ver averagely correlated with coeff: 0.67.
          - scaled _variance and scaled_radius_of_gyration, circularity & distance_circularity also seems to be highly correlated with corr coeff: 0.79
          - pr.axis_recatngularity and max.length_recatngularity also seems to be strongly correlated with coeff: 0.81 
          - scatter_ratio and elongatedness seems to be have strong negative correlation val : 0.97
          - elongatedness and pr.axis_rectangularity seems to have strong negative correlation, val:  0.95
       
          
   
### Little To No Correlation: 
          -  
          -max_length_aspect_ratio & radius_ratio have average correlation with coeff: 0.5
          - pr.axis_aspect_ratio & max_length_aspect_ratio seems to have very little correlation
          - scaled_radius_gyration & scaled_radisu_gyration.1 seems to be very little correlated
          - scaled_radius_gyration.1 & skewness_about seems to be very little correlated
          - skewness_about & skewness_about.1 not be correlated
          - skewness_about.1 and skewness_about.2 are not correlated.
        
let's visualize the same with pairplot , to see how it looks visually . 

        
## Pairplot Analysis:           

In [ ]:
sns.pairplot(cleandf, diag_kind="kde")

## Quick Insights:
As observed in our correlation heatmap our pairplot seems to validate the same. Scaled Variance & Scaled Variance.1 seems to be have very strong positive correlation with value of 0.98. skewness_about_2 and hollow_ratio also seems to have strong positive correation with coeff: 0.89

scatter_ratio and elongatedness seems to be have very strong negative correlation. elongatedness and pr.axis_rectangularity seems to have strong negative correlation with val of 

 
We found from our pairplot analysis that, Scaled Variance & Scaled Variance.1 and elongatedness and pr.axis_rectangularity to be strongly correlated , so they need to dropped of treated carefully before we go for model building. 

## Choosing the right attributes which can be the right choice for model building

since our objective is to reocgnize whether an object is a van or bus or car based on some input features. so our main assumption is there is little or no multicollinearity between the features. if our dataset has perfectly positive or negative attributes as can be obseverd fro our correlation analysis, there is a high chance that the performance of the model will be impacted by a problem called — “Multicollinearity”. Multicollinearity happens when one predictor variable in a multiple regression model can be linearly predicted from the others with a high degree of accuracy. This can lead to skewed or misleading results.

if two features is highly correlated then there is no point using both features.in that case, we can drop one feature. SNS heatmap gives us the correlation matrix where we can see which features are highly correlated. 

From above correlation matrix we can see that there are many features which are highly correlated. if we carefully analyse, we will find that many  features are there which having more than 0.9 correlation. so we can decide to get rid of those columns whose correlation is +-0.9 or above.There are 8 such columns:

- max.length_rectangularity
- scaled_radius_of_gyration
- skewness_about.2
- scatter_ratio
- elongatedness
- pr.axis_rectangularity
- scaled_variance
- scaled_variance.1

### Outcome: 
Also we observed that more than 50 % of our attributes ar highly correlated , so  what we can we do best to deal with this kind problem of Multicollinearity

### Well, There are multiple ways to deal with this problem. The easiest way is to delete or eliminate one of the perfectly correlated features.  We can pick one of the tiwo highly correalated variables and drop another one. like in our case Scaled Variance & Scaled Variance.1 are having strong positive correlation , so we can pick one and drop one as they will only make our dimension redundant.

### Similarly between elongatedness and pr.axis_rectangularity we can pick one as they have very strong negative correlation. This approach can be used to select the feature we want to carry forward for model analysis. But there is another better approach called PCA

### Another method, is to use a dimension reduction algorithm such as Principle Component Analysis (PCA). We will go for PCA and analyse the same going forward: 


In [ ]:
#display how many are car,bus,van. 
newdf['class'].value_counts()

splitscaledf = newdf.copy()
sns.countplot(newdf['class'])
plt.show()


# Principal Component Analysis(PCA):

     - Basically PCA is a dimension redcuction methodology which aims to reduce a large set of (often correlated) variables into a smaller set of (uncorrelated) variables, called principal components, which holds sufficient information without loosing the the relevant info much.
     
     - Principal components are new variables that are constructed as linear combinations or mixtures of the initial variables. These combinations are done in such a way that the new variables (i.e., principal components) are uncorrelated and most of the information within the initial variables is squeezed or compressed into the first components.
     
## We will perform PCA in following steps:
   - Split our data into train and test data set
   - normalize the tarining set using standard scalar
   - Calculate the covariance matrix.
   - Calculate the eigenvectors and their eigenvalues.
   - Sort the eigenvectors according to their eigenvalues in descending order.
   - Choose the first K eigenvectors (where k is the dimension we'd like to end up with).
   - Build new dataset with reduced dimensionality.
   

# Approach 1: 

### Separate The Data Into Independent & Dependent attribute


In [ ]:
#now separate the dataframe into dependent and independent variables
#X1= newdf.drop('class',axis=1)
#y1 = newdf['class']
#print("shape of new_vehicle_df_independent_attr::",X.shape)
#print("shape of new_vehicle_df_dependent_attr::",y.shape)

X = newdf.iloc[:,0:18].values
y = newdf.iloc[:,18].values

X

### Scaling The Independent Data Set: 

In [ ]:
from sklearn.preprocessing import StandardScaler
#We transform (centralize) the entire X (independent variable data) to normalize it using standardscalar through transformation. We will create the PCA dimensions
# on this distribution. 
sc = StandardScaler()
X_std =  sc.fit_transform(X)          


## Calculating covariance matrix:

Covariance matrix should be 18*18 matrix


In [ ]:

cov_matrix = np.cov(X_std.T)
print("cov_matrix shape:",cov_matrix.shape)
print("Covariance_matrix",cov_matrix)

## Calculating Eigen Vectors & Eigen Values: Using numpy linear algebra function


In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
print('Eigen Vectors \n%s', eigenvectors)
print('\n Eigen Values \n%s', eigenvalues)

## Sort eigenvalues in descending order

In [ ]:

# Make a set of (eigenvalue, eigenvector) pairs:

eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

# Sort the (eigenvalue, eigenvector) pairs from highest to lowest with respect to eigenvalue
eig_pairs.sort()

eig_pairs.reverse()
print(eig_pairs)

# Extract the descending ordered eigenvalues and eigenvectors
eigvalues_sorted = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sorted = [eig_pairs[index][1] for index in range(len(eigenvalues))]

# Let's confirm our sorting worked, print out eigenvalues
print('Eigenvalues in descending order: \n%s' %eigvalues_sorted)

In [ ]:
tot = sum(eigenvalues)
var_explained = [(i / tot) for i in sorted(eigenvalues, reverse=True)]  # an array of variance explained by each 
# eigen vector... there will be 18 entries as there are 18 eigen vectors)
cum_var_exp = np.cumsum(var_explained)  # an array of cumulative variance. There will be 18 entries with 18 th entry 
# cumulative reaching almost 100%

## Plotting The Explained Variance and Princiapl Components:  

In [ ]:

plt.bar(range(1,19), var_explained, alpha=0.5, align='center', label='individual explained variance')
plt.step(range(1,19),cum_var_exp, where= 'mid', label='cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc = 'best')
plt.show()

### Quick Observation: 
    - From above we plot we can clealry observer that 8 dimension() are able to explain 95 %variance of data. 
    - so we will use first 8 principal components going forward and calulate the reduced dimensions. 

## Dimensionality Reduction

Now 8 dimensions seems very reasonable. With 8 variables we can explain over 95% of the variation in the original data!

In [ ]:
# P_reduce represents reduced mathematical space....

P_reduce = np.array(eigvectors_sorted[0:8])   # Reducing from 8 to 4 dimension space

X_std_8D = np.dot(X_std,P_reduce.T)   # projecting original data into principal component dimensions

reduced_pca = pd.DataFrame(X_std_8D)  # converting array to dataframe for pairplot

reduced_pca

### Let us check The Pairplot Of Reduced Dimension After PCA: 

In [ ]:

sns.pairplot(reduced_pca, diag_kind='kde') 
#sns.pairplot(reduced_pca1, diag_kind='kde') 

### It is clealry visible from the pairplot above that:
After dimensionality reduction using PCA our attributes have become independent with no correlation among themselves. As most of them have cloud of data points with no lienaer kind of relationship.

## Fitting Model and measuring score simply on Original Data : 

### Fit SVC Model ON Train-test Data: 

Let's build two Support Vector Classifier Model one with 18 original independent variables and the second one with only the 8 new reduced variables constructed using PCA.

In [ ]:
#now split the data into 70:30 ratio

#orginal Data
Orig_X_train,Orig_X_test,Orig_y_train,Orig_y_test = train_test_split(X_std,y,test_size=0.30,random_state=1)

#PCA Data
pca_X_train,pca_X_test,pca_y_train,pca_y_test = train_test_split(reduced_pca,y,test_size=0.30,random_state=1)
#pca_X_train,pca_X_test,pca_y_train,pca_y_test = train_test_split(reduced_pca1,y,test_size=0.30,random_state=1)

# Let's train the model with both original data and pca data with new dimension

### Fitting SVC model On Original Data

In [ ]:

svc = SVC() #instantiate the object


In [ ]:
#fit the model on orighinal raw data
svc.fit(Orig_X_train,Orig_y_train)

In [ ]:
#predict the y value
Orig_y_predict = svc.predict(Orig_X_test)



### Fitting SVC ON PCA Data: 

In [ ]:
#now fit the model on pca data with new dimension
svc1 = SVC() #instantiate the object
svc1.fit(pca_X_train,pca_y_train)

#predict the y value
pca_y_predict = svc1.predict(pca_X_test)

In [ ]:
#display accuracy score of both models

print("Model Score On Original Data ",svc.score(Orig_X_test, Orig_y_test))
print("Model Score On Reduced PCA Dimension ",svc1.score(pca_X_test, pca_y_test))

print("Before PCA On Original 18 Dimension",accuracy_score(Orig_y_test,Orig_y_predict))
print("After PCA(On 8 dimension)",accuracy_score(pca_y_test,pca_y_predict))

### Quick Observation: 
   - On training data set we saw that our support vector classifier without performing PCA has an accuracy score of 95 % 
   - But when we applied the SVC model on PCA componenets(reduced dimensions) our model scored 93 %. 
   - Considering that original dataframe had 18 dimensions and After PCA dimension reduced to 8, our model has fared well in terms of accuracy score. 

### Confusion Matrix: 


In [ ]:
# Calculate Confusion Matrix & PLot To Visualize it

def draw_confmatrix(y_test, yhat, str1, str2, str3, datatype ):
    #Make predictions and evalute
    #model_pred = fit_test_model(model,X_train, y_train, X_test)
    cm = confusion_matrix( y_test, yhat, [0,1,2] )
    print("Confusion Matrix For :", "\n",datatype,cm )
    sns.heatmap(cm, annot=True,  fmt='.2f', xticklabels = [str1, str2,str3] , yticklabels = [str1, str2,str3] )
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    

draw_confmatrix(Orig_y_test, Orig_y_predict,"Van ", "Car ", "Bus", "Original Data Set" )

draw_confmatrix(pca_y_test, pca_y_predict,"Van ", "Car ", "Bus", "For Reduced Dimensions Using PCA ")

#Classification Report Of Model built on Raw Data
print("Classification Report For Raw Data:", "\n", classification_report(Orig_y_test,Orig_y_predict))

#Classification Report Of Model built on Principal Components:

print("Classification Report For PCA:","\n", classification_report(pca_y_test,pca_y_predict))

### Quick Comments: 

### Confusion Metric Analysis ON Original  Data: 

Confusion Matrix For : 
 Original Data Set 
 [[ 58   0   1]
 [  1 129   3]
 [  6   1  55]]
     
       - Our model on original data set has correctly classified 58 van out of 59 actuals vans and has errored only in one case where it has wrongly predicted van to be a bus.  
       - IN case of 133 actual cars our svm model has correcly classified 129 cars. it has wrongly classified 3 cars to be a bus and also 1 car to be a van
       - In case of 62 instances of actual bus , our model has correctly classified 55 buses , It has faltered in classifying wrongly 6 buses to be a van and one bus to be a car. 
       
       
### Confusion Metric Analysis ON Reduced Dimesnion After PCA : 

For Reduced Dimensions Using PCA:
 [[ 57   2   0]
 [  2 126   5]
 [  1   7  54]]
    
    - Out of 59 actual instances of vans our model has correctly predicted 57 vans and errored in 2 instances where it wrongly classified vans to be a car. 
    - Out of 133 actuals cars , our mdoel has correclty classified 126 of them to be a  car and faltered in 7 cases where it wrongly classified 5 cars to a bus and 2 cars to be a van. 
    
    - Out of 62 actual bus , our model has correclty classified 54 of them to be a bus. It has faltered in 8 cases where it wrongly classified 7 bus to be a car and 1 bus to be a van.
    
# Let's See The Classification Report Metrics:

Classification Report For Raw Data: 
               precision    recall  f1-score   support

         0.0       0.89      0.98      0.94        59
         1.0       0.99      0.97      0.98       133
         2.0       0.93      0.89      0.91        62

   micro avg       0.95      0.95      0.95       254
   macro avg       0.94      0.95      0.94       254
weighted avg       0.95      0.95      0.95       254

Classification Report For PCA: 
               precision    recall  f1-score   support

         0.0       0.95      0.97      0.96        59
         1.0       0.93      0.95      0.94       133
         2.0       0.92      0.87      0.89        62

   micro avg       0.93      0.93      0.93       254
   macro avg       0.93      0.93      0.93       254
weighted avg       0.93      0.93      0.93       254


### Insights On Classification Reports: 

### On original data: 
   
     - our model has 99 % precison  score when it comes to classify car from the given set of silhoutte parameters. It has 89 % precision when it comes to classifying the input as van, while it has 93 % precison when it come to predict data as bus. 
     
     - In term of recall score our model has recal score of 98 % for van classification, 97 % for car and 89 % for bus. 
     - OUr model has an weighted average of 95 % for all classification metrics. 
   
### On Reduced Dimensions After PCA: 

     - Our model has highest precision score of 95 % when it comes to predict van type, which is better as compared to predcition done on original data set, which came out with the precision score of 89 % for van. 
     - Recall score is almost neck to neck with what our model scored on original data set. It showed highest recall score of 97 % in classifying data as car. 

# Approach 2: 

# Let's Perfrom The PCA and See How Our Model Perform:

### When we split our data set into test train and then apply scaling. 
   
In previous  process of PCA and model performance comparison, we scaled our data before splitting them into train and test, which may have lead to some kind of data leakages. Let's see how our PCA process and model accuracy behaves when we scale our data after splitting the into train-test set 
   

In [ ]:
splitscaledf.head(850)

In [ ]:
splitscale_X = splitscaledf.iloc[:,0:18].values
splitscale_y = splitscaledf.iloc[:,18].values

print("Indpendent Variable X",splitscale_X )
print("Class Variable y",splitscale_y )

## Let's Split Our Data Into Test & Train Data Set

In [ ]:
#splitting the data in train and test sets into 70:30 Ratio

SplitScale_X_train, SplitScale_X_test, SplitScale_y_train, SplitScale_y_test = train_test_split(splitscale_X,splitscale_y, test_size = 0.3, random_state = 10)

## Let's Apply StandardScaler to normalize our data set

In [ ]:
ssx_train_sd = StandardScaler().fit_transform(SplitScale_X_train)
ssx_test_sd = StandardScaler().fit_transform(SplitScale_X_test)

print(len(ssx_train_sd))
print(len(ssx_test_sd))


## Calculating covariance matrix:

Covariance matrix should be 18*18 matrix

In [ ]:
# generating the covariance matrix and the eigen values for the PCA analysis
cov_matrix_1 = np.cov(ssx_train_sd.T) # the relevanat covariance matrix
print('Covariance Matrix \n%s', (cov_matrix_1))

#generating the eigen values and the eigen vectors
e_vals, e_vecs = np.linalg.eig(cov_matrix_1)
print('Eigenvectors \n%s' %(e_vecs))
print('\nEigenvalues \n%s' %e_vals)

### Sort eigenvalues in descending order

In [ ]:
# Step 3 (continued): Sort eigenvalues in descending order

# Make a set of (eigenvalue, eigenvector) pairs
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

# Sort the (eigenvalue, eigenvector) pairs from highest to lowest with respect to eigenvalue
eig_pairs.sort()

eig_pairs.reverse()
print(eig_pairs)

# Extract the descending ordered eigenvalues and eigenvectors
eigvalues_sorted = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sorted = [eig_pairs[index][1] for index in range(len(eigenvalues))]

# Let's confirm our sorting worked, print out eigenvalues
print('Eigenvalues in descending order: \n%s' %eigvalues_sorted)

In [ ]:
tot = sum(eigenvalues)
var_explained = [(i / tot) for i in sorted(eigenvalues, reverse=True)]  # an array of variance explained by each 
# eigen vector... there will be 8 entries as there are 8 eigen vectors)
cum_var_exp = np.cumsum(var_explained)  # an array of cumulative variance. There will be 8 entries with 8 th entry 
# cumulative reaching almost 100%

### Visualizing The plot : Principal Componenet Vs Explained Variance Ratio

In [ ]:
plt.bar(range(1,19), var_explained, alpha=0.5, align='center', label='individual explained variance')
plt.step(range(1,19),cum_var_exp, where= 'mid', label='cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc = 'best')
plt.show()

# Dimensionality Reduction:  

In [ ]:
# P_reduce represents reduced mathematical space....

P_reduce_1 = np.array(eigvectors_sorted[0:8])   # Reducing from 8 to 4 dimension space

X_train_std_pca = np.dot(ssx_train_sd,P_reduce_1.T)   # projecting original data into principal component dimensions

X_test_std_pca = np.dot(ssx_test_sd,P_reduce_1.T) 
#Proj_data_df_new = pd.DataFrame(X_std_8D_1) 

print(X_train_std_pca)
print(X_test_std_pca)

Projected_df_train = pd.DataFrame(X_train_std_pca)
Projected_df_test = pd.DataFrame(X_test_std_pca)

### Pairplot Analysis : On Training PCA Data Set

In [ ]:
sns.pairplot(Projected_df_train, diag_kind='kde')


In [ ]:
### Pairplot Analysis : On Test PCA Data Set

In [ ]:
sns.pairplot(Projected_df_test, diag_kind='kde')

### Quick Insights: 

    - We can see that after we performed PCA our new dataframe with reduced dimesnions has no to zero linear relationship among themseleves, which is the main objective of using PCA tool. Almost all attribures have cloud of data in the mathematical space with no clear positive or negative correlation.
    

In [ ]:
ssx_train_sd.shape, P_reduce_1.T.shape, X_train_std_pca.shape, X_test_std_pca.shape

In [ ]:
clf1 = SVC()
clf1.fit(ssx_train_sd, SplitScale_y_train)
print ('Before PCA score', clf1.score(ssx_test_sd, SplitScale_y_test))

clf2 = SVC()
clf2.fit(X_train_std_pca, SplitScale_y_train)
print ('After PCA score', clf2.score(X_test_std_pca, SplitScale_y_test))


#print("Before PCA On Original 18 Dimension",accuracy_score(Orig_y_test,Orig_y_predict))
#print("After PCA(On 8 dimension)",accuracy_score(pca_y_test,pca_y_predict))

#predict the y value
pca_yhat_predict= clf2.predict(X_test_std_pca)

#orginal data yhat value
orig_yhat_predict = clf1.predict(ssx_test_sd)

print("Before PCA On Original 18 Dimension",accuracy_score(SplitScale_y_test,orig_yhat_predict))
print("After PCA(On 8 dimension)",accuracy_score(SplitScale_y_test,pca_yhat_predict))

### Quick Observation: After we split the data set into test & train set before applying standardscaler():

   - On training data set we saw that our support vector classifier without performing PCA has an accuracy score of 95 % 
   - But when we applied the SVC model on PCA componenets(reduced dimensions) our model still performed better with 95 % with only miniscule fall in score. 
   - So earlier when we scaled the data first and then use it to build our model we saw a 1-2 % difference in accuracy score of svc model build on orignal & pca data set respectively. But when we changed our strategy by splitting the data set into train-test first and then using it scale the train & test data set we avoided the possible data leakage, which resulted in very minor change in accuracy score, when model was built both on original data set & pca data set. 
   
   - Considering that original dataframe had 18 dimensions and After PCA dimension reduced to 8, our model has fared well in terms of accuracy score. 

# Calculate Confusion Matrix & Plot To Visualize it

In [ ]:

draw_confmatrix(SplitScale_y_test, orig_yhat_predict,"Van ", "Car ", "Bus", "Original Data Set" )

draw_confmatrix(SplitScale_y_test, pca_yhat_predict,"Van ", "Car ", "Bus", "For Reduced Dimensions Using PCA ")

#Classification Report Of Model built on Raw Data
print("Classification Report For Raw Data:", "\n", classification_report(SplitScale_y_test,orig_yhat_predict))

#Classification Report Of Model built on Principal Components:

print("Classification Report For PCA:","\n", classification_report(SplitScale_y_test,pca_yhat_predict))

## Quick Insights On Confusion Matrix: 
         
     - In our approach 1: we saw that our model actual instances of Van : 59, car : 133 and bus : 62   
     - But in our second approach where we split the data set and then scaled , we saw the actual categorization instances as Van : 71, Car : 125, Bus: 58 
     
## Confusion Matrix For : 

### Original Data Set:
 
 [[ 68   1   2]
 [  1 118   6]
 [  0   1  57]]
   
     and 
     
 For Reduced Dimensions Using PCA 
 [[ 68   3   0]
 [  2 118   5]
 [  1   1  56]]
 
 
- Clearly both the model has almost same level of precitiability when it comes to correctly classifying van , car and bus. Which was not the case with approach 1. 

- IN approach one our confuson matrix showed that model on original data out of 133 actual car instance, correctly predicted 129 instances to be cars. while in appraich to out of actual 125 cars , it has correctly predicted 118 of instances to be a car . 


## Let's Apply Grid Search & Cross-Validation:To Tune Our Model and Validate The Model's Accuracy Score

In [ ]:
import itertools

def classifiers_hypertune(name,rf,param_grid,x_train_scaled,y_train,x_test_scaled,y_test,CV):
    CV_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=CV, verbose= 1, n_jobs =-1 )
    CV_rf.fit(x_train_scaled, y_train)
    
    y_pred_train = CV_rf.predict(x_train_scaled)
    y_pred_test = CV_rf.predict(x_test_scaled)
    
    print('Best Score: ', CV_rf.best_score_)
    print('Best Params: ', CV_rf.best_params_)
    
    
    
    #Classification Report
    print(name+" Classification Report: ")
    print(classification_report(y_test, y_pred_test))
    
   
    #Confusion Matrix for test data
    draw_confmatrix(y_test, y_pred_test,"Van", "Car", "Bus", "Original Data Set" )
    print("SVM Accuracy Score:",round(accuracy_score(y_test, y_pred_test),2)*100)
    

In [ ]:

#Training on SVM Classifier
from sklearn.model_selection import GridSearchCV
svmc = SVC()

#Let's See What all parameters one can tweak 
print("SVM Parameters:", svmc.get_params())

# Create the parameter grid based on the results of random search 
param_grid = [
  {'C': [0.01, 0.05, 0.5, 1], 'kernel': ['linear']},
  {'C': [0.01, 0.05, 0.5, 1],  'kernel': ['rbf']},
 ]

param_grid_1 = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

## Hypertuning SVM using hyper Parameters: 

### Iteration 1: 

### In Case Of PCA: 


In [ ]:
classifiers_hypertune("Support Vector Classifier", svmc, param_grid,X_train_std_pca, SplitScale_y_train, X_test_std_pca, SplitScale_y_test,10)

### Quick Insight: 

### Hyper tuning of Model on PCA Data Set: 

   - We generally tune some of the important hyperparameters of the model which are not the model parameter. like here we played with C value : penalty and the type of kernel : rbf/liner  
   
   - GridSearchCV get's the best paramter from the array of parameters and find the best model and score for us. 
   - Here on perfroming gridsearch hyper tuning of SVM model we got the best parameter to  be 
    : Best Score:  0.9256756756756757
    : Best Params:  {'C': 1, 'kernel': 'rbf'}
    : Accruacy Score : 95 % which seems to be similar to than what we measured earlier in our appraoch 2
    
    - We can further play with hyper parameters and see if it helps to score our model better. 

### For Original Data Set: 

In [ ]:
classifiers_hypertune("Support Vector Classifier", svmc, param_grid,ssx_train_sd, SplitScale_y_train, ssx_test_sd, SplitScale_y_test,10)


### On Orginal Data Set: 

   - When we apploed GridSearchCV on Our model which is using orginal data set, we saw
      - Best Score:  0.956081081081081
      - Best Params:  {'C': 1, 'kernel': 'rbf'}
      - Accuracy Score : We saw an improvement in accuracy score to  96 % which is better than what we observed in approach 2 and approach 1


# Let's further tweak the parameters to see if we can improve our model accuracy :

In [ ]:
classifiers_hypertune("Support Vector Classifier_iterarion2", svmc, param_grid_1,X_train_std_pca, SplitScale_y_train, X_test_std_pca, SplitScale_y_test,10)

### Iteration 2 : On original Data Set

In [ ]:
classifiers_hypertune("Support Vector Classifier", svmc, param_grid_1,ssx_train_sd, SplitScale_y_train, ssx_test_sd, SplitScale_y_test,10)

### We can see a slight improvement in best model which was picked by our gridsearchcv method: 96.79 % and also we saw a slight increase in model accuracy score : 97 % 

## Closing Comments: 

    - So we can see that how Principal component analysis can help us cherry pick only the relevant features by analysing the relationship between independent attributes to see which one will be more helpful in building our model without introducing any impurity in it. 
    
    - We can increase the models performance both in terms of processing speed and the ease of implementing it . 
    
    - Also we learned that one should always make sure data is normalized/scaled before perfroming PCA, else the result will not be as per expectations
    - Also it is recommended though it's not a rule , to split your data set into test-train before you apply data normalization/scaling. This will help our model to perfrom more realistically in production environment. 
    - If one scale the model before splitting it into train-test , there is a high chance of data leakage . By data leakage i mean that our model will be alaready influenced by test data , as it has tasted a bit of test data even before it will fit and tested on it. 
    
    - Also once one have tried every bit of techniques of removing outliers, treating the missing value, normalizing  the data whenever required, applied the diffrent models to come up with better model based on metrics which suits more the problem statement. It is advisable to fine tune it using hyperparameter tuning techniques which tunes the model performances and also employs Cross-fold validation internally to make sure our model is ready to face production environment. 
    
- It would be good if we also measure how other classifier model like logistic regression, decisiontree , emsemble model like randomforrestclassifier will perfrom both on original data & PCA data. It would be interesting to see the confusion matrix and various others metrics like recall score, precision score, f1-score etc. 

- But Since our objective was to see how PCA data set impacts the model perfromance specially on training data set , we have here achived our objective and saw that PCA is great tool to make our model perfrom better . 

## Let's Look How Some Other Classifier Models Perform Bith On Original Data & PCA treated data sets

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(ssx_train_sd, SplitScale_y_train)
print ('Before PCA score', model.score(ssx_test_sd, SplitScale_y_test))

model.fit(X_train_std_pca, SplitScale_y_train)
print ('After PCA score', model.score(X_test_std_pca, SplitScale_y_test))



### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(ssx_train_sd, SplitScale_y_train)
print ('Before PCA score', nb.score(ssx_test_sd, SplitScale_y_test))

nb.fit(X_train_std_pca, SplitScale_y_train)
print ('After PCA score', nb.score(X_test_std_pca, SplitScale_y_test))


### Decisiontree Classifier: 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier(criterion = 'entropy' )

dt_model.fit(ssx_train_sd, SplitScale_y_train)
print ('Before PCA score', dt_model.score(ssx_test_sd, SplitScale_y_test))

dt_model.fit(X_train_std_pca, SplitScale_y_train)
print ('After PCA score', dt_model.score(X_test_std_pca, SplitScale_y_test))


## It Seems that Support Vectore Classifier is a better model to classifiy the given silhoutte info as van, bus, car . 